## Environment Setting

In [106]:
import pandas as pd
import numpy as np

In [107]:
import multiprocessing

num_processors = multiprocessing.cpu_count()
num_processors

20

In [108]:
from pandarallel import pandarallel

# Initialize pandarallel
pandarallel.initialize(nb_workers = multiprocessing.cpu_count()-1, use_memory_fs=False, progress_bar=False)

INFO: Pandarallel will run on 19 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [109]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## Data Loading

In [110]:
%%time
df_raw = pd.read_parquet('news_final_project.parquet', engine='pyarrow')

CPU times: total: 4.53 s
Wall time: 7.8 s


In [111]:
df_raw.head(1)

url        date language  \
0  http://en.people.cn/n3/2021/0318/c90000-9830122.html  2021-03-18       en   

                                                                                           title  \
0  Artificial intelligence improves parking efficiency in Chinese cities - People's Daily Online   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [112]:
df_raw.shape

(200332, 5)

## Data Cleaning

In [113]:
import re

In [114]:
# filter english article
df_raw = df_raw[df_raw.language == 'en']
df_raw.shape

(200332, 5)

#### URL cleaning

In [116]:
df_raw.url[:5]

0                                                                                                                                                                                                                                                                                    http://en.people.cn/n3/2021/0318/c90000-9830122.html
1                                                                                                                                                                                                http://newsparliament.com/2020/02/27/children-with-autism-saw-their-learning-and-social-skills-boosted-after-playing-with-this-ai-robot/
2                                                                                                                                                                                                                                                                                                        http://www.dataweek.co.za/12835r
3         

In [118]:
def extract_domain(url):
    from urllib.parse import urlparse
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    return domain

In [119]:
df_raw['domain'] = df_raw['url'].parallel_apply(extract_domain)

In [120]:
df_raw['domain'].head()

0                                 en.people.cn
1                           newsparliament.com
2                           www.dataweek.co.za
3    www.homeoffice.consumerelectronicsnet.com
4                        www.itbusinessnet.com
Name: domain, dtype: object

#### Title Cleaning

In [121]:
df_raw.title.head(10)

0                                                                                                                                         Artificial intelligence improves parking efficiency in Chinese cities - People's Daily Online
1                                                                                                                  Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot – News Parliament
2                                                                                                            Forget ML, AI and Industry 4.0 – obsolescence should be your focus - 26 February 2021 - Test & Rework Solutions - Dataweek
3                                                                                                                            Strategy Analytics: 71% of Smartphones Sold Globally in 2021 will be AI Powered – Consumer Electronics Net
4                                                                       

In [122]:
def clean_title(title):
    import re

    cleaned_title = re.split(r'[–|]', title)[0].strip()

    return cleaned_title

In [123]:
df_raw['title'] = df_raw['title'].str.replace('-', '–')

In [124]:
df_raw['clean_title'] = df_raw['title'].apply(clean_title)

In [125]:
df_raw.clean_title.head(10)

0                                                                    Artificial intelligence improves parking efficiency in Chinese cities
1                                       Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot
2                                                                                                           Forget ML, AI and Industry 4.0
3                                                          Strategy Analytics: 71% of Smartphones Sold Globally in 2021 will be AI Powered
4                Olympus to Support Endoscopic AI Diagnosis Education for Doctors in India and to Launch AI Diagnostic Support Application
5                                                                      Cr Bard Inc Has Returned 48.9% Since SmarTrend Recommendation (BCR)
6                                                       From the Bard to broadcaster: Stratford Festival builds new identity with streamer
7                          

#### Text Cleaning

In [126]:
from IPython.display import Markdown as md

In [127]:
df_raw.text[3]

'\n\nStrategy Analytics: 71% of Smartphones Sold Globally in 2021 will be AI Powered – Consumer Electronics Net\n \nSkip to content\n\nConsumer Electronics Net\n\nPrimary Menu\n\nConsumer Electronics Net\n\nSearch for:\n \nHomeNewsStrategy Analytics: 71% of Smartphones Sold Globally in 2021 will be AI Powered \n \n                                 News\n                             \n \nStrategy Analytics: 71% of Smartphones Sold Globally in 2021 will be AI Powered\n                    7 hours ago            \n \n\nArtificial Intelligence Now Powers the Majority of Smartphones\n\nBOSTON–(BUSINESS WIRE)–Strategy Analytics in a newly published report, Smartphones: Global Artificial Intelligence Technologies Forecast to 2025, finds that on-device Artificial Intelligence (AI) is being rapidly implemented by smartphone vendors. AI is used in various functions inside smartphones such as intelligent power optimization, imaging, virtual assistants, and to enhance device performance. The report 

Found that the first part of text are always title + some description of the article website such as language settings, author name, etc. After the second occurence of the title, there are the true text of the article.

Therefore, we could use this characteristic to exclude the first useless part of the text.

In [128]:
equal_irrelevant = [


    'comments section',
    'privacy policy',
    'Contact us',
    'Join us',
    'Photos',
    'Home',
    'No results found',
    'Report a problem',
    'Related articles',
 
]

# previously I removed all the elements (may be paragraphs of the main text seprarated by a new line) contains those words
# and therefore may remove so some articles that cotain keywords in those removed paragraphs
# So finally there are only 118k articles and after fixing this problem, there are 168k


In [129]:
# include irrelevant words --> remove
include_irrelevant = [
    'http',
    'www',
    '.com',
    'subscribe to our', 
    'Photo taken',
    'press release',
    'Connect with us',
    'Follow us on instagram',
    'Follow us on twitter', 
    'Like us on facebook',
    'Email us at',
    'Copyright @',
    'Copyright at',
    '.org',
    'You may have missed',
    'Leave a reply',
    'Related posts',
    'photo credit',
    'terms of service',
    'all rights reserved',
    'click here to',
    'follow us on',
    'sign up for',
    'editor’s note',
    'Share this article',
    'Image source',
    'Join our mailing list',
    'Sponsored content',


    # the below is second cleaning after NER
    'The Associated Press',
    'When typing in this field a list of search results',
    'Gray Media Group',
    'Digi Communications'
   
]

In [130]:

recommended_phrases = [
    "you may have missed",
    "read more",
    "related articles",
    "recommended for you",
    "also read",
    "explore more",
    "don't miss",
    'do not miss',
    "similar stories",
    "trending now",
    "more stories",
    "related Stories",
    'more information and articles',
    'sponsored content',
    'Editor’s pick',
    'Matched content'

]



##### Test

In [58]:
test = df_raw.text[0]

In [20]:
elements = re.split('\r|\n|\t|\/|\|', test)
elements

['',
 '',
 "Artificial intelligence improves parking efficiency in Chinese cities - People's Daily Online",
 '',
 'Home',
 'China Politics',
 'Foreign Affairs',
 'Opinions',
 'Video: We Are China',
 'Business',
 'Military',
 'World',
 'Society',
 'Culture',
 'Travel',
 'Science',
 'Sports',
 'Photo',
 '',
 'Languages',
 '',
 'Chinese',
 'Japanese',
 'French',
 'Spanish',
 'Russian',
 'Arabic',
 'Korean',
 'German',
 'Portuguese',
 'Thursday, March 18, 2021',
 'Home>>',
 '',
 '',
 '',
 'Artificial intelligence improves parking efficiency in Chinese cities',
 "By Liu\xa0Shiyao (People's Daily) 09:16, March 18, 2021",
 'Photo taken on July 1, 2019, shows a sign for electronic toll collection (ETC) newly set up at a roadside parking space on Yangzhuang road, Shijingshan district, Beijing. Some urban areas of the city started to use ETC system for roadside parking spaces since July 1, 2019. (People’s Daily Online',
 'Li Wenming)',
 '',
 '',
 'Thanks to the application of an artificial intel

In [21]:
len(elements)

99

In [22]:
elements = [el for el in elements if len(el) > 0]


# now the list contains main text, some long sentence of irrelanvent instructions, "read others"
# remove elements that contains anything in irrelanvent
elements = [el for el in elements if not any(irrelevant_word in el for irrelevant_word in irrelevant)]

In [23]:
elements

["Artificial intelligence improves parking efficiency in Chinese cities - People's Daily Online",
 'China Politics',
 'Foreign Affairs',
 'Opinions',
 'Video: We Are China',
 'Business',
 'Military',
 'World',
 'Society',
 'Culture',
 'Travel',
 'Science',
 'Sports',
 'Photo',
 'Languages',
 'Chinese',
 'Japanese',
 'French',
 'Spanish',
 'Russian',
 'Arabic',
 'Korean',
 'German',
 'Portuguese',
 'Thursday, March 18, 2021',
 'Artificial intelligence improves parking efficiency in Chinese cities',
 "By Liu\xa0Shiyao (People's Daily) 09:16, March 18, 2021",
 'Li Wenming)',
 'Thanks to the application of an artificial intelligence (AI)-empowered roadside electronic toll collection (ETC) system, China’s capital city Beijing has seen significant improvement in the efficiency of parking fee collection, turnover of roadside parking spots, order in roadside parking, as well as traffic congestion.',
 'As the city further deepens its roadside parking reform, the ETC system has almost covered al

In [24]:
len(elements)

46

In [25]:
elements = [el for el in elements if len(el.split()) > 10 and len(el) > 50]

In [26]:
elements

["Artificial intelligence improves parking efficiency in Chinese cities - People's Daily Online",
 'Thanks to the application of an artificial intelligence (AI)-empowered roadside electronic toll collection (ETC) system, China’s capital city Beijing has seen significant improvement in the efficiency of parking fee collection, turnover of roadside parking spots, order in roadside parking, as well as traffic congestion.',
 'As the city further deepens its roadside parking reform, the ETC system has almost covered all the roadside parking spaces in the city, with the proportion of vehicles parked on roads using the system exceeding 90 percent.',
 'With the AI-empowered system, drivers can park their vehicles at the parking spots on the roadside, and then pay the parking charge via their mobile phones after they drive away.',
 '“Since the summer of 2019, roadside ETC devices have been installed here. With all the cars being parked in designated parking spots on the roadside, the road now s

In [27]:
len(elements)

12

In [28]:
elements = [el for el in elements if df_raw.clean_title[0] not in el]

In [29]:
elements

['Thanks to the application of an artificial intelligence (AI)-empowered roadside electronic toll collection (ETC) system, China’s capital city Beijing has seen significant improvement in the efficiency of parking fee collection, turnover of roadside parking spots, order in roadside parking, as well as traffic congestion.',
 'As the city further deepens its roadside parking reform, the ETC system has almost covered all the roadside parking spaces in the city, with the proportion of vehicles parked on roads using the system exceeding 90 percent.',
 'With the AI-empowered system, drivers can park their vehicles at the parking spots on the roadside, and then pay the parking charge via their mobile phones after they drive away.',
 '“Since the summer of 2019, roadside ETC devices have been installed here. With all the cars being parked in designated parking spots on the roadside, the road now seems brighter and wider,” Wang said.',
 'The smart roadside ETC system “AIpark Sky Eye” adopted by

In [30]:
# Join the elements back together
filtered_text = ' '.join(elements)
filtered_text

"Thanks to the application of an artificial intelligence (AI)-empowered roadside electronic toll collection (ETC) system, China’s capital city Beijing has seen significant improvement in the efficiency of parking fee collection, turnover of roadside parking spots, order in roadside parking, as well as traffic congestion. As the city further deepens its roadside parking reform, the ETC system has almost covered all the roadside parking spaces in the city, with the proportion of vehicles parked on roads using the system exceeding 90 percent. With the AI-empowered system, drivers can park their vehicles at the parking spots on the roadside, and then pay the parking charge via their mobile phones after they drive away. “Since the summer of 2019, roadside ETC devices have been installed here. With all the cars being parked in designated parking spots on the roadside, the road now seems brighter and wider,” Wang said. The smart roadside ETC system “AIpark Sky Eye” adopted by Beijing is devel

In [31]:
filtered_text = re.sub(r'[\[|\(][^\]|\)]*?(?=http)', '', filtered_text)
filtered_text

"Thanks to the application of an artificial intelligence (AI)-empowered roadside electronic toll collection (ETC) system, China’s capital city Beijing has seen significant improvement in the efficiency of parking fee collection, turnover of roadside parking spots, order in roadside parking, as well as traffic congestion. As the city further deepens its roadside parking reform, the ETC system has almost covered all the roadside parking spaces in the city, with the proportion of vehicles parked on roads using the system exceeding 90 percent. With the AI-empowered system, drivers can park their vehicles at the parking spots on the roadside, and then pay the parking charge via their mobile phones after they drive away. “Since the summer of 2019, roadside ETC devices have been installed here. With all the cars being parked in designated parking spots on the roadside, the road now seems brighter and wider,” Wang said. The smart roadside ETC system “AIpark Sky Eye” adopted by Beijing is devel

In [32]:
filtered_text = re.sub(r'<a.*?>', '', filtered_text)
filtered_text

"Thanks to the application of an artificial intelligence (AI)-empowered roadside electronic toll collection (ETC) system, China’s capital city Beijing has seen significant improvement in the efficiency of parking fee collection, turnover of roadside parking spots, order in roadside parking, as well as traffic congestion. As the city further deepens its roadside parking reform, the ETC system has almost covered all the roadside parking spaces in the city, with the proportion of vehicles parked on roads using the system exceeding 90 percent. With the AI-empowered system, drivers can park their vehicles at the parking spots on the roadside, and then pay the parking charge via their mobile phones after they drive away. “Since the summer of 2019, roadside ETC devices have been installed here. With all the cars being parked in designated parking spots on the roadside, the road now seems brighter and wider,” Wang said. The smart roadside ETC system “AIpark Sky Eye” adopted by Beijing is devel

In [33]:
len(filtered_text)

3035

In [34]:
filtered_text = re.sub(r'\bpic.twitter.com/\S+', '', filtered_text)
filtered_text
len(filtered_text)

3035

In [35]:
filtered_text = re.sub(r'\bpic.twitter.com/\S+', '', filtered_text)
len(filtered_text)

3035

In [36]:
filtered_text = re.sub(r'#[\S]+\b', '', filtered_text)
len(filtered_text)

3035

In [37]:
filtered_text = re.sub(r'\b@\S+\b', '', filtered_text)
len(filtered_text)

3035

In [38]:
filtered_text = re.sub(r'[\w\.-]+@[\w\.-]+\.\w+', '', filtered_text)
len(filtered_text)

3035

In [39]:
filtered_text = re.sub(r'http\S+', '', filtered_text)
len(filtered_text)

3035

In [40]:
filtered_text = re.sub(r'[^\w\s]', '', filtered_text)
filtered_text

'Thanks to the application of an artificial intelligence AIempowered roadside electronic toll collection ETC system Chinas capital city Beijing has seen significant improvement in the efficiency of parking fee collection turnover of roadside parking spots order in roadside parking as well as traffic congestion As the city further deepens its roadside parking reform the ETC system has almost covered all the roadside parking spaces in the city with the proportion of vehicles parked on roads using the system exceeding 90 percent With the AIempowered system drivers can park their vehicles at the parking spots on the roadside and then pay the parking charge via their mobile phones after they drive away Since the summer of 2019 roadside ETC devices have been installed here With all the cars being parked in designated parking spots on the roadside the road now seems brighter and wider Wang said The smart roadside ETC system AIpark Sky Eye adopted by Beijing is developed operated and maintaine

In [41]:
filtered_text = re.sub(r'\s+', ' ', filtered_text)
len(filtered_text)

2972

##### Formal Cleaning

In [131]:
def clean_news(text, clean_title):
    # import re


    # First, split the text by new lines, tab, |, -
    elements = re.split('\r|\n|\t|\/|\|', text)

    # remove empty elements --> solve \n\n situations
    elements = [el for el in elements if len(el) > 0]


   # remove the article recommendations
    for i, el in enumerate(elements):
        if el.lower() in recommended_phrases:
            elements = elements[:i]
            break

    # now the list contains main text, some long sentence of irrelanvent instructions, "read others"
    # remove elements that contains anything in irrelevant
    elements = [el for el in elements if not any(irrelevant_word.lower() in el.lower() for irrelevant_word in include_irrelevant)]


    # remove elements that equal to annything in irrelevant --> actually unnecessary since the length  control will work for it
    # elements = [el for el in elements if not any(el.lower() == irrelevant_word.lower() for irrelevant_word in equal_irrelevant)]


    # now filter out elements that are likely not part of the main text --> but some main text consists of paragraphs separated by \n
    # so we cannot set a threshold for main text. Instead we should set a threshold for each paragraph
    # Also, the length of paragraph for news is usually much shorter
    elements = [el for el in elements if len(el.split()) > 10 and len(el) > 50]

    # escape the elemet contaning the title
    elements = [el for el in elements if clean_title not in el]

    # Join the elements back together
    filtered_text = ' '.join(elements)


    # Then, proceed with the some basic clean-up steps: some should be removed but I just kept them in case

    # remove remnants of web crawls
    filtered_text = re.sub(r'[\[|\(][^\]|\)]*?(?=http)', '', filtered_text)
    filtered_text = re.sub(r'<a.*?>', '', filtered_text)
    filtered_text = re.sub(r'\bpic.twitter.com/\S+', '', filtered_text)
    filtered_text = re.sub(r'#[\S]+\b', '', filtered_text)
    filtered_text = re.sub(r'\b@\S+\b', '', filtered_text)
    filtered_text = re.sub(r'[\w\.-]+@[\w\.-]+\.\w+', '', filtered_text)
    # remove URLs
    filtered_text = re.sub(r'http\S+', '', filtered_text)
    # no need to remove punctuation and special characters
    # filtered_text = re.sub(r'[^\w\s]', '', filtered_text)
    # remove extra white spaces
    filtered_text = re.sub(r'\s+', ' ', filtered_text)
    
    return filtered_text


In [132]:
%%time
df_raw['clean_text'] = df_raw.apply(lambda x: clean_news(x['text'], x['clean_title']), axis=1)

CPU times: total: 2min 54s
Wall time: 5min 23s


In [70]:
# df_raw['clean_text'] = df_raw.parallel_apply(lambda x: clean_news(x['text'], x['clean_title']), axis=1)

In [133]:
df_raw.clean_text[0]

"Thanks to the application of an artificial intelligence (AI)-empowered roadside electronic toll collection (ETC) system, China’s capital city Beijing has seen significant improvement in the efficiency of parking fee collection, turnover of roadside parking spots, order in roadside parking, as well as traffic congestion. As the city further deepens its roadside parking reform, the ETC system has almost covered all the roadside parking spaces in the city, with the proportion of vehicles parked on roads using the system exceeding 90 percent. With the AI-empowered system, drivers can park their vehicles at the parking spots on the roadside, and then pay the parking charge via their mobile phones after they drive away. “This road used to be full of cars, and even the normal lanes were occupied. You could hardly move a bit during the morning and evening commute time,” recalled a citizen surnamed Wang, who lives in Chaoyang district of Beijing. “Since the summer of 2019, roadside ETC devices

#### Check after First NER

In [ ]:
# check whether digital communications is a news source
digi_rows = df_raw[df_raw['clean_text'].str.contains('Digi Communications', case=False)]
digi_rows.shape

(183, 7)

In [ ]:
digi_rows.url.head()

80      https://express-press-release.net/news/2022/11/07/1287823
2098    https://express-press-release.net/news/2023/04/14/1401010
5152    https://express-press-release.net/news/2022/04/06/1138245
6194    https://express-press-release.net/news/2022/04/14/1144652
9240    https://express-press-release.net/news/2021/09/25/1023059
Name: url, dtype: object

In [59]:
gray_media_rows = df_raw[df_raw['clean_text'].str.contains('Gray Media Group Inc', case=False)]
gray_media_rows.shape

(26956, 7)

In [60]:
gray_media_rows.url.head()

280                                                     https://www.1011now.com/prnewswire/2022/11/03/kbr-awarded-analytic-modeling-data-science-contracts-worth-over-120m/
286                                  https://www.actionnews5.com/prnewswire/2022/07/29/quantum-music-showcase-groundbreaking-ai-baby-crying-translator-q-bear-tta-pavilion/
287                          https://www.actionnews5.com/prnewswire/2023/03/21/manufacturers-focused-digital-transformation-through-ai-recognized-by-symphonyai-industrial/
288    https://www.alaskasnewssource.com/prnewswire/2022/08/02/workwave-launches-servicebot-by-workwave-delivering-powerful-ai-sales-technology-variety-service-industries/
289                                               https://www.alaskasnewssource.com/prnewswire/2023/04/04/homedigy-launches-geodrops-worlds-smartest-ai-irrigation-manager/
Name: url, dtype: object

## Filter Relevant News

In [134]:
# filter relevant text
keywords = ['artificial intelligence', 'data science', 'machine Learning', 'AI']

df_filtered = df_raw[df_raw['clean_text'].str.contains('|'.join(keywords), case=False)]

In [135]:
df_filtered = df_filtered.reset_index(drop=True)

In [136]:
df_filtered = df_filtered.drop(['url', 'language'], axis=1)


In [137]:
df_filtered.head(1)

date  \
0  2021-03-18   

                                                                                           title  \
0  Artificial intelligence improves parking efficiency in Chinese cities – People's Daily Online   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [138]:
df_filtered.shape

(165530, 6)

## Tokenization, Lemmatization, and Concatnation

In [139]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [140]:
# Download necessary resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to C:\Users\Eason
[nltk_data]     Peng\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Eason
[nltk_data]     Peng\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Eason
[nltk_data]     Peng\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [141]:
def clean_tokens(text):
    
    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove single-character tokens (mostly punctuation)
    tokens = [word for word in tokens if len(word) > 1]

    # Do not remove numbers --> numbers here may matter
    # tokens = [word for word in tokens if not word.isnumeric()]

    # Remove punctuation
    tokens = [word for word in tokens if word.isalpha()]

    # Lowercase all words (default_stopwords are lowercase too)
    tokens = [word.lower() for word in tokens]
    
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    
    lemmatized_tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]
    return " ".join(lemmatized_tokens)

In [142]:
%%time
df_filtered['clean_token'] = df_filtered['clean_text'].apply(clean_tokens)

CPU times: total: 4min 44s
Wall time: 9min 48s


## Save to Local

In [145]:
df_filtered.columns

Index(['date', 'title', 'text', 'domain', 'clean_title', 'clean_text',
       'clean_token'],
      dtype='object')

In [146]:
df_filtered = df_filtered.drop(['text', 'title'], axis=1)

In [147]:
df_filtered.head(1)

date        domain  \
0  2021-03-18  en.people.cn   

                                                             clean_title  \
0  Artificial intelligence improves parking efficiency in Chinese cities   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [148]:
df_filtered.shape

(165530, 5)

In [149]:
%%time
# save filtered df
df_filtered.to_parquet('filtered_data.parquet', engine='pyarrow')

CPU times: total: 2.86 s
Wall time: 6.44 s
